# Molecular Descriptor Summary and Analysis

This notebook focuses on calculating molecular descriptors, analyzing their distributions, and performing correlation analysis to understand the chemical space of BBB permeability.

## Objectives:
- Calculate comprehensive molecular descriptors
- Analyze descriptor distributions for permeable vs non-permeable molecules
- Perform correlation analysis between descriptors
- Identify key descriptors for BBB permeability
- Visualize chemical space and descriptor relationships

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sys
import os

# Add src directory to path
sys.path.append('../src')

from data_handler import DataHandler
from descriptors import MolecularDescriptors
from feature_engineering import FeatureEngineering

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

print("Libraries imported successfully!")

## 1. Load Clean Dataset

In [ ]:
# Load the clean dataset from previous notebook
try:
    df = pd.read_csv('../data/BBBP_clean.csv')
    print(f"Clean dataset loaded: {df.shape}")
except FileNotFoundError:
    print("Clean dataset not found. Loading and cleaning original dataset...")
    data_handler = DataHandler()
    df = data_handler.load_bbbp_data()
    # Basic cleaning
    valid_indices = []
    for idx, smiles in enumerate(df['smiles']):
        is_valid, mol = data_handler.validate_smiles(smiles)
        if is_valid:
            valid_indices.append(idx)
    df = df.iloc[valid_indices].drop_duplicates(subset=['smiles']).reset_index(drop=True)
    print(f"Dataset cleaned: {df.shape}")

print(f"\nClass distribution:")
print(df['p_np'].value_counts())
df.head()

## 2. Calculate Molecular Descriptors

In [ ]:
# Initialize descriptor calculator
descriptor_calc = MolecularDescriptors()

print("Calculating molecular descriptors...")
print("This may take a few minutes for the full dataset...")

# Calculate descriptors for all molecules
descriptors_list = []
failed_calculations = []

for idx, smiles in enumerate(df['smiles']):
    try:
        descriptors = descriptor_calc.calculate_all_descriptors(smiles)
        descriptors['compound_idx'] = idx
        descriptors_list.append(descriptors)
    except Exception as e:
        failed_calculations.append((idx, smiles, str(e)))
        print(f"Failed to calculate descriptors for compound {idx}: {e}")

# Create descriptors dataframe
descriptors_df = pd.DataFrame(descriptors_list)
descriptors_df = descriptors_df.set_index('compound_idx')

print(f"\nDescriptors calculated for {len(descriptors_df)} compounds")
print(f"Failed calculations: {len(failed_calculations)}")
print(f"Descriptor features: {descriptors_df.shape[1]}")

# Merge with original data
df_with_descriptors = df.join(descriptors_df, how='inner')
print(f"Final dataset shape: {df_with_descriptors.shape}")

# Display descriptor columns
descriptor_columns = descriptors_df.columns.tolist()
print(f"\nCalculated descriptors: {descriptor_columns}")

## 3. Descriptor Statistics and Overview

In [ ]:
# Basic descriptor statistics
print("=== DESCRIPTOR STATISTICS ===")
descriptor_stats = descriptors_df.describe()
print(descriptor_stats)

# Check for missing values in descriptors
print("\n=== MISSING VALUES IN DESCRIPTORS ===")
missing_values = descriptors_df.isnull().sum()
if missing_values.sum() > 0:
    print(missing_values[missing_values > 0])
else:
    print("No missing values found in descriptors")

# Descriptor ranges and distributions
print("\n=== DESCRIPTOR RANGES ===")
for col in descriptor_columns:
    min_val = descriptors_df[col].min()
    max_val = descriptors_df[col].max()
    mean_val = descriptors_df[col].mean()
    std_val = descriptors_df[col].std()
    print(f"{col:15s}: {min_val:8.3f} - {max_val:8.3f} (mean: {mean_val:6.3f}, std: {std_val:6.3f})")

## 4. Descriptor Distributions by BBB Permeability

In [ ]:
# Compare descriptor distributions between permeable and non-permeable molecules
permeable = df_with_descriptors[df_with_descriptors['p_np'] == 1]
non_permeable = df_with_descriptors[df_with_descriptors['p_np'] == 0]

print(f"Permeable molecules: {len(permeable)}")
print(f"Non-permeable molecules: {len(non_permeable)}")

# Statistical comparison
print("\n=== STATISTICAL COMPARISON (t-test) ===")
statistical_results = []

for descriptor in descriptor_columns:
    perm_values = permeable[descriptor].dropna()
    non_perm_values = non_permeable[descriptor].dropna()
    
    # Perform t-test
    t_stat, p_value = stats.ttest_ind(perm_values, non_perm_values)
    
    # Calculate effect size (Cohen's d)
    pooled_std = np.sqrt(((len(perm_values) - 1) * perm_values.var() + 
                         (len(non_perm_values) - 1) * non_perm_values.var()) / 
                        (len(perm_values) + len(non_perm_values) - 2))
    cohens_d = (perm_values.mean() - non_perm_values.mean()) / pooled_std
    
    statistical_results.append({
        'descriptor': descriptor,
        'permeable_mean': perm_values.mean(),
        'non_permeable_mean': non_perm_values.mean(),
        't_statistic': t_stat,
        'p_value': p_value,
        'cohens_d': cohens_d,
        'significant': p_value < 0.05
    })

stats_df = pd.DataFrame(statistical_results)
stats_df = stats_df.sort_values('p_value')

print("Top 10 most significant descriptors:")
print(stats_df.head(10)[['descriptor', 'permeable_mean', 'non_permeable_mean', 'p_value', 'cohens_d']])

## 5. Descriptor Distribution Visualizations

In [ ]:
# Plot distributions for key descriptors
key_descriptors = ['LogP', 'MW', 'TPSA', 'HBD', 'HBA', 'RotBonds']
available_descriptors = [desc for desc in key_descriptors if desc in descriptor_columns]

if len(available_descriptors) >= 4:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.ravel()
    
    for i, descriptor in enumerate(available_descriptors[:4]):
        ax = axes[i]
        
        # Plot histograms
        ax.hist(non_permeable[descriptor].dropna(), alpha=0.6, label='Non-permeable', 
                bins=30, color='lightcoral', density=True)
        ax.hist(permeable[descriptor].dropna(), alpha=0.6, label='Permeable', 
                bins=30, color='lightblue', density=True)
        
        ax.set_xlabel(descriptor)
        ax.set_ylabel('Density')
        ax.set_title(f'{descriptor} Distribution')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print(f"Available descriptors for plotting: {available_descriptors}")
    # Plot available descriptors
    for descriptor in available_descriptors:
        plt.figure(figsize=(10, 6))
        plt.hist(non_permeable[descriptor].dropna(), alpha=0.6, label='Non-permeable', 
                bins=30, color='lightcoral', density=True)
        plt.hist(permeable[descriptor].dropna(), alpha=0.6, label='Permeable', 
                bins=30, color='lightblue', density=True)
        plt.xlabel(descriptor)
        plt.ylabel('Density')
        plt.title(f'{descriptor} Distribution')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

## 6. Box Plots for Key Descriptors

In [ ]:
# Create box plots for comparison
if len(available_descriptors) >= 6:
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()
    
    for i, descriptor in enumerate(available_descriptors[:6]):
        ax = axes[i]
        
        # Prepare data for box plot
        data_to_plot = [non_permeable[descriptor].dropna(), permeable[descriptor].dropna()]
        
        box_plot = ax.boxplot(data_to_plot, labels=['Non-permeable', 'Permeable'], 
                             patch_artist=True)
        
        # Color the boxes
        box_plot['boxes'][0].set_facecolor('lightcoral')
        box_plot['boxes'][1].set_facecolor('lightblue')
        
        ax.set_ylabel(descriptor)
        ax.set_title(f'{descriptor} by BBB Permeability')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    # Create individual box plots
    for descriptor in available_descriptors:
        plt.figure(figsize=(8, 6))
        data_to_plot = [non_permeable[descriptor].dropna(), permeable[descriptor].dropna()]
        box_plot = plt.boxplot(data_to_plot, labels=['Non-permeable', 'Permeable'], 
                              patch_artist=True)
        box_plot['boxes'][0].set_facecolor('lightcoral')
        box_plot['boxes'][1].set_facecolor('lightblue')
        plt.ylabel(descriptor)
        plt.title(f'{descriptor} by BBB Permeability')
        plt.grid(True, alpha=0.3)
        plt.show()

## 7. Correlation Analysis

In [ ]:
# Calculate correlation matrix
correlation_matrix = descriptors_df.corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8}, fmt='.2f')
plt.title('Molecular Descriptor Correlation Matrix')
plt.tight_layout()
plt.show()

# Find highly correlated descriptor pairs
print("\n=== HIGHLY CORRELATED DESCRIPTOR PAIRS (|r| > 0.7) ===")
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.7:
            high_corr_pairs.append({
                'descriptor1': correlation_matrix.columns[i],
                'descriptor2': correlation_matrix.columns[j],
                'correlation': corr_val
            })

if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs)
    high_corr_df = high_corr_df.sort_values('correlation', key=abs, ascending=False)
    print(high_corr_df)
else:
    print("No highly correlated pairs found (|r| > 0.7)")

## 8. Correlation with BBB Permeability

In [ ]:
# Calculate correlation between descriptors and BBB permeability
permeability_correlations = []

for descriptor in descriptor_columns:
    corr_coef = df_with_descriptors[descriptor].corr(df_with_descriptors['p_np'])
    permeability_correlations.append({
        'descriptor': descriptor,
        'correlation': corr_coef,
        'abs_correlation': abs(corr_coef)
    })

perm_corr_df = pd.DataFrame(permeability_correlations)
perm_corr_df = perm_corr_df.sort_values('abs_correlation', ascending=False)

print("=== CORRELATION WITH BBB PERMEABILITY ===")
print(perm_corr_df)

# Plot correlation with permeability
plt.figure(figsize=(12, 8))
colors = ['red' if x < 0 else 'blue' for x in perm_corr_df['correlation']]
bars = plt.barh(range(len(perm_corr_df)), perm_corr_df['correlation'], color=colors, alpha=0.7)
plt.yticks(range(len(perm_corr_df)), perm_corr_df['descriptor'])
plt.xlabel('Correlation with BBB Permeability')
plt.title('Descriptor Correlation with BBB Permeability')
plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
plt.grid(True, alpha=0.3)

# Add correlation values as text
for i, (bar, corr) in enumerate(zip(bars, perm_corr_df['correlation'])):
    plt.text(corr + (0.01 if corr >= 0 else -0.01), i, f'{corr:.3f}', 
             ha='left' if corr >= 0 else 'right', va='center')

plt.tight_layout()
plt.show()

## 9. Principal Component Analysis (PCA)

In [ ]:
# Perform PCA on descriptors
feature_eng = FeatureEngineering()

# Scale descriptors
scaled_descriptors = feature_eng.scale_features(descriptors_df)

# Perform PCA
pca = PCA()
pca_result = pca.fit_transform(scaled_descriptors)

# Calculate explained variance
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

print(f"Number of components: {len(explained_variance_ratio)}")
print(f"Variance explained by first 5 components: {cumulative_variance[:5]}")

# Plot explained variance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Scree plot
ax1.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, 'bo-')
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Explained Variance Ratio')
ax1.set_title('PCA Scree Plot')
ax1.grid(True, alpha=0.3)

# Cumulative variance plot
ax2.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'ro-')
ax2.axhline(y=0.95, color='k', linestyle='--', alpha=0.7, label='95% variance')
ax2.set_xlabel('Number of Components')
ax2.set_ylabel('Cumulative Explained Variance')
ax2.set_title('Cumulative Explained Variance')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find number of components for 95% variance
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"\nNumber of components needed for 95% variance: {n_components_95}")

## 10. Chemical Space Visualization

In [ ]:
# Plot chemical space using first two principal components
plt.figure(figsize=(12, 8))

# Create scatter plot colored by permeability
permeable_mask = df_with_descriptors['p_np'] == 1
non_permeable_mask = df_with_descriptors['p_np'] == 0

plt.scatter(pca_result[non_permeable_mask, 0], pca_result[non_permeable_mask, 1], 
           c='lightcoral', alpha=0.6, label='Non-permeable', s=50)
plt.scatter(pca_result[permeable_mask, 0], pca_result[permeable_mask, 1], 
           c='lightblue', alpha=0.6, label='Permeable', s=50)

plt.xlabel(f'PC1 ({explained_variance_ratio[0]:.1%} variance)')
plt.ylabel(f'PC2 ({explained_variance_ratio[1]:.1%} variance)')
plt.title('Chemical Space Visualization (PCA)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print component loadings for interpretation
print("\n=== TOP LOADINGS FOR FIRST TWO PRINCIPAL COMPONENTS ===")
feature_names = descriptors_df.columns

# PC1 loadings
pc1_loadings = pd.DataFrame({
    'descriptor': feature_names,
    'loading': pca.components_[0]
}).sort_values('loading', key=abs, ascending=False)

print("\nPC1 (most important loadings):")
print(pc1_loadings.head(5))

# PC2 loadings
pc2_loadings = pd.DataFrame({
    'descriptor': feature_names,
    'loading': pca.components_[1]
}).sort_values('loading', key=abs, ascending=False)

print("\nPC2 (most important loadings):")
print(pc2_loadings.head(5))

## 11. Summary and Key Insights

In [ ]:
# Generate comprehensive summary
print("=== MOLECULAR DESCRIPTOR ANALYSIS SUMMARY ===")
print(f"\nDataset: {len(df_with_descriptors)} compounds")
print(f"Descriptors calculated: {len(descriptor_columns)}")
print(f"Permeable compounds: {len(permeable)} ({len(permeable)/len(df_with_descriptors)*100:.1f}%)")
print(f"Non-permeable compounds: {len(non_permeable)} ({len(non_permeable)/len(df_with_descriptors)*100:.1f}%)")

print("\n=== KEY FINDINGS ===")

# Most discriminative descriptors
top_discriminative = stats_df.head(3)
print("\nMost discriminative descriptors (lowest p-values):")
for _, row in top_discriminative.iterrows():
    direction = "higher" if row['permeable_mean'] > row['non_permeable_mean'] else "lower"
    print(f"- {row['descriptor']}: Permeable molecules have {direction} values (p={row['p_value']:.2e})")

# Strongest correlations with permeability
top_correlations = perm_corr_df.head(3)
print("\nStrongest correlations with BBB permeability:")
for _, row in top_correlations.iterrows():
    direction = "positive" if row['correlation'] > 0 else "negative"
    print(f"- {row['descriptor']}: {direction} correlation (r={row['correlation']:.3f})")

# PCA insights
print(f"\nPCA Analysis:")
print(f"- First 2 components explain {cumulative_variance[1]:.1%} of variance")
print(f"- {n_components_95} components needed for 95% variance")
print(f"- PC1 mainly driven by: {pc1_loadings.iloc[0]['descriptor']} (loading: {pc1_loadings.iloc[0]['loading']:.3f})")
print(f"- PC2 mainly driven by: {pc2_loadings.iloc[0]['descriptor']} (loading: {pc2_loadings.iloc[0]['loading']:.3f})")

# Highly correlated descriptors
if len(high_corr_pairs) > 0:
    print(f"\nHighly correlated descriptor pairs: {len(high_corr_pairs)}")
    print("Consider removing redundant descriptors for model training")

print("\n=== RECOMMENDATIONS ===")
print("- Focus on top discriminative descriptors for model building")
print("- Consider feature selection to remove highly correlated descriptors")
print("- Use PCA for dimensionality reduction if needed")
print("- Proceed to notebook 03_model_training.ipynb for machine learning")

# Save descriptor data for next notebook
df_with_descriptors.to_csv('../data/BBBP_with_descriptors.csv', index=False)
print("\nDataset with descriptors saved to '../data/BBBP_with_descriptors.csv'")